# Data pipeline from Spotify
Se realiza un seguimiento de las canciones de spotify escuhcadas desde el día de ayer

## Modulos y paquetes

In [3]:
# Importaciones
import requests
import datetime
import pandas as pd
from sqlalchemy import Column, String, Integer, DateTime, Date, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
import pytz # to time zone

## Constantes

In [2]:
# Constants
DATA_BASE_LOCATION = "sqlite:///my_tracks.db"
USER_ID = 'guido.simoca'
TOKEN = 'token de seguridad' # https://developer.spotify.com/console/get-recently-played/

# Extraction

## Configuraciones de requests

In [4]:
# url 
url = "https://api.spotify.com/v1/me/player/recently-played"
# header for the request acording to spotify
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {TOKEN}"
    }

# Defining the datetime to track
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days = 1)
yesterday_unix = int(yesterday.timestamp())*1000 # To convert in unix tamstamp
# query
query = {'limit': 50, 'after': yesterday_unix}

# Requests
r = requests.get(url, params=query, headers=headers)

# Data
data = r.json()

## Obtencion de los datos

In [5]:
# Fields
played_at = [i['played_at'] for i in data['items']]
timestamps = [i[:10] for i in played_at]
artist_name = [i['track']['album']['artists'][0]['name'] for i in data['items']]
song_name = [i['track']['name'] for i in data['items']]

## Conversion a DataFrame

In [6]:
# Convert to dic
song_dic = {
    'played_at': played_at,
    'timestamps': timestamps,
    'artist_name': artist_name,
    'song_name': song_name
}

# Convert to DataFrame
song_df = pd.DataFrame(song_dic)
song_df

,played_at,timestamps,artist_name,song_name
0,2022-11-07T12:11:29.074Z,2022-11-07,DJ Alex,TU AMOR | E7
1,2022-11-07T12:09:09.178Z,2022-11-07,Rusherking,Perfecta
2,2022-11-07T12:06:30.042Z,2022-11-07,Alan Gomez,CALLEJERO FINO | Mission 10
3,2022-11-07T12:03:44.002Z,2022-11-07,Rei,Tu Turrito
4,2022-11-07T12:00:07.130Z,2022-11-07,La Joaqui,Butakera


# Transformacion y Validacion

In [7]:
# Validation of the dataset
def check_if_valid_data(data: pd.DataFrame) -> bool:
    # Check if the dataset is empty
    if data.empty:
        print('No song downloaded. Finishing excecution') # Empty datasets means that I dont hear any songs
        return False # No means an error!

    # Primary Key check (Duplicate check)
    if not(data['played_at'].is_unique):
        raise Exception("Primary Key check is violated")
    
    # Null values check
    if song_df.isnull().values.any():
        raise Exception("There are some nulls values")
    
    # Timestamp check
    convert_to_datetime_date = lambda x: datetime.datetime.strptime(x[:10], "%Y-%m-%d").date()
    day = song_df['played_at'].agg(convert_to_datetime_date)
    if (day < yesterday.date()).any():
        raise Exception("There is at least one song that is not from yesterday")

check_if_valid_data(song_df)

# Carga de datas

## Creacion de la tabla

In [8]:
# Defining the class to inherit the class for create the table
Base = declarative_base()

# Defining the class to create the table
class Played_list(Base):
    __tablename__ = 'played_list'
    id = Column(Integer(), primary_key = True, nullable = False, unique = True)
    played_at = Column(DateTime(25),  nullable = False, unique = True)
    timestamps = Column(Date(), nullable = False)
    artist_name = Column(String(80), nullable = False)
    song_name = Column(String(80), nullable = False)
    date_created = Column(DateTime(), default = datetime.datetime.now(pytz.timezone('America/Argentina/Buenos_Aires')))

    # To print
    def __repr__(self):
        
        return f"""Song:
        ID = {self.id}
        played_at = {self.played_at}
        timestamps = {self.timestamps}
        artist_name = {self.artist_name}
        song_name = {self.song_name}
        date_created = {self.date_created}"""

In [9]:
# funtion to convert the date elements from the dataframe to datetime object
def conver_played_at_to_datetime(date: str) -> object:
    return datetime.datetime.strptime(date[:-5], "%Y-%m-%dT%H:%M:%S")

conver_played_at_to_datetime(song_df['played_at'][0])

datetime.datetime(2022, 11, 7, 12, 11, 29)

In [10]:
# Object for prototype
song = Played_list(played_at = conver_played_at_to_datetime(song_df['played_at'][0]),
                   timestamps = datetime.datetime.strptime(song_df['timestamps'][0],"%Y-%m-%d").date(),
                   artist_name = song_df['artist_name'][0],
                   song_name = song_df['song_name'][0],
)

song

Song:
        ID = None
        played_at = 2022-11-07 12:11:29
        timestamps = 2022-11-07
        artist_name = DJ Alex
        song_name = TU AMOR | E7
        date_created = None

In [11]:
# Creating engine
engine = create_engine(DATA_BASE_LOCATION, echo = True)

# creating data base
Base.metadata.create_all(engine)

2022-11-07 10:55:42,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-07 10:55:42,904 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("played_list")
2022-11-07 10:55:42,905 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-07 10:55:42,908 INFO sqlalchemy.engine.Engine COMMIT


# Guardando los datos

### Opcion 1
- Directo de dataframe a data base
- Se cargan todos los registros de una vez

In [12]:
# Con esta opcion no se carga la fecha de insercion, ya que esta se genera en el momento del add
# Tiene mas sentido si la tabla no fue creada con anterioridad con ORM
song_df2 = pd.DataFrame(song_dic)
song_df2['played_at'] = song_df2['played_at'].agg(lambda x: conver_played_at_to_datetime(x))
song_df2['timestamps'] = song_df2['timestamps'].agg(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
# Se debe agregar el tiempo de carga
song_df2['date_created'] = datetime.datetime.now(pytz.timezone('America/Argentina/Buenos_Aires'))
song_df2.to_sql('played_list',engine, index = False, if_exists = 'append', method = 'multi')

2022-11-07 10:55:49,108 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("played_list")
2022-11-07 10:55:49,109 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-07 10:55:49,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-07 10:55:49,121 INFO sqlalchemy.engine.Engine INSERT INTO played_list (played_at, timestamps, artist_name, song_name, date_created) VALUES (?, ?, ?, ?, ?), (?, ?, ?, ?, ?), (?, ?, ?, ?, ?), (?, ?, ?, ?, ?), (?, ?, ?, ?, ?)
2022-11-07 10:55:49,122 INFO sqlalchemy.engine.Engine [no key 0.00127s] ('2022-11-07 12:11:29.000000', '2022-11-07', 'DJ Alex', 'TU AMOR | E7', '2022-11-07 10:55:49.100455', '2022-11-07 12:09:09.000000', '2022-11-07', 'Rusherking', 'Perfecta', '2022-11-07 10:55:49.100455', '2022-11-07 12:06:30.000000', '2022-11-07', 'Alan Gomez', 'CALLEJERO FINO | Mission 10', '2022-11-07 10:55:49.100455', '2022-11-07 12:03:44.000000', '2022-11-07', 'Rei', 'Tu Turrito', '2022-11-07 10:55:49.100455', '2022-11-07 12:00:07.000000', '2022-11-07', 'La Joa

5

 ### Opcion 2
 - De dataframe a objeto y recien a db
 - Se carga registro por registro


In [17]:
# Declarate session
Session = sessionmaker()
local_session = Session(bind = engine)

# Loading de data
for i in song_df.values:
    print(i)
    song = Played_list(
                            played_at = conver_played_at_to_datetime(i[0]),
                            timestamps = datetime.datetime.strptime(i[1],"%Y-%m-%d").date(),
                            artist_name = i[2],
                            song_name = i[3]
                     )

    local_session.add(song)
local_session.commit()

['2022-11-07T12:11:29.074Z' '2022-11-07' 'DJ Alex' 'TU AMOR | E7']
['2022-11-07T12:09:09.178Z' '2022-11-07' 'Rusherking' 'Perfecta']
['2022-11-07T12:06:30.042Z' '2022-11-07' 'Alan Gomez'
 'CALLEJERO FINO | Mission 10']
['2022-11-07T12:03:44.002Z' '2022-11-07' 'Rei' 'Tu Turrito']
['2022-11-07T12:00:07.130Z' '2022-11-07' 'La Joaqui' 'Butakera']
2022-11-07 11:17:35,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-07 11:17:35,906 INFO sqlalchemy.engine.Engine INSERT INTO played_list (played_at, timestamps, artist_name, song_name, date_created) VALUES (?, ?, ?, ?, ?)
2022-11-07 11:17:35,908 INFO sqlalchemy.engine.Engine [cached since 1277s ago] ('2022-11-07 12:11:29.000000', '2022-11-07', 'DJ Alex', 'TU AMOR | E7', '2022-11-07 10:55:36.774808')
2022-11-07 11:17:35,915 INFO sqlalchemy.engine.Engine INSERT INTO played_list (played_at, timestamps, artist_name, song_name, date_created) VALUES (?, ?, ?, ?, ?)
2022-11-07 11:17:35,916 INFO sqlalchemy.engine.Engine [cached since 1277s ago